In [6]:
import numpy as np
import pandas as pd
import librosa as lr
import glob
import os
from pyroomacoustics.experimental.rt60 import measure_rt60

from IPython.display import Audio

In [2]:
rir_df = pd.read_csv('0812_rir_df.csv')

In [3]:
sr = 16000
rir_sample, _ = lr.load(rir_df.iloc[200]['file'], sr=sr)

In [4]:
Audio(data=rir_sample, rate=sr)

In [5]:
rt60_sample = measure_rt60(rir_sample, fs=sr, decay_db=60)

In [6]:
rt60_sample

13.362

In [7]:
def compute_rt60(rir_path, sr=16000):
    rir, _ = lr.load(rir_path, sr=sr)
    rt60 = measure_rt60(rir, fs=sr)
    return rt60

In [8]:
compute_rt60(rir_df.iloc[200]['file'])

13.362

In [9]:
files = rir_df['file'].tolist()
rt60_dict = {}
bad_files = []
for file in files:
    try:
        rt60 = compute_rt60(file)
    except:
        print(file)
        bad_files.append(file)
        continue
    rt60_dict[file] = rt60

/scratch/ci411/sonos_rirs/rirs/openair/alcuin-college-university-york/b-format/alcuin_s4r1_bformat.wav
/scratch/ci411/sonos_rirs/rirs/openair/alcuin-college-university-york/b-format/s4r1_spist_bform.wav
/scratch/ci411/sonos_rirs/rirs/openair/alcuin-college-university-york/b-format/s3r1_spist_bform.wav
/scratch/ci411/sonos_rirs/rirs/openair/ron-cooke-hub-university-york/b-format/sssr.wav
/scratch/ci411/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID06_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/scratch/ci411/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID08_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/scratch/ci411/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID03_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/scratch/ci411/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID05_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/scratch/ci411/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID04_20180105_

In [10]:
rt60_list = []
for row in rir_df.iterrows():
    file = row[1]['file']
    try:
        rt60 = compute_rt60(file)
    except:
        print(file)
        rt60 = None
        pass
    rt60_list.append(rt60)
rir_df['computed_rt60'] = rt60_list

/scratch/ci411/sonos_rirs/rirs/openair/alcuin-college-university-york/b-format/alcuin_s4r1_bformat.wav
/scratch/ci411/sonos_rirs/rirs/openair/alcuin-college-university-york/b-format/s4r1_spist_bform.wav
/scratch/ci411/sonos_rirs/rirs/openair/alcuin-college-university-york/b-format/s3r1_spist_bform.wav
/scratch/ci411/sonos_rirs/rirs/openair/ron-cooke-hub-university-york/b-format/sssr.wav
/scratch/ci411/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID06_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/scratch/ci411/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID08_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/scratch/ci411/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID03_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/scratch/ci411/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID05_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/scratch/ci411/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID04_20180105_

In [12]:
rooms = rir_df['room'].unique()
gb_room = rir_df.groupby('room')
mean_rt60_dict = {}
for room in rooms:
    mean_rt60 = gb_room.get_group(room)['computed_rt60'].mean()
    mean_rt60_dict[room] = mean_rt60

ace_Building_Lobby
ace_Office_1
ace_Meeting_Room_2
ace_Meeting_Room_1
ace_Office_2
ace_Lecture_Room_1
ace_Lecture_Room_2
air_booth
air_lecture
air_meeting
air_office
alcuin-college-university-york
arthur-sykes-rymer-auditorium-university-york
central-hall-university-york
dixon-studio-theatre-university-york
falkland-palace-royal-tennis-court
genesis-6-studio-live-room-drum-set
hamilton-mausoleum
heslington-church-vaa-group-2
innocent-railway-tunnel
maes-howe
r1-nuclear-reactor-hall
ron-cooke-hub-university-york
shrine-and-parish-church-all-saints-north-street-_
sports-centre-university-york
spring-lane-building-university-york
st-andrews-church
st-margarets-church-national-centre-early-music
st-patricks-church-patrington
terrys-factory-warehouse
terrys-typing-room
tyndall-bruce-monument
usina-del-arte-symphony-hall
york-guildhall-council-chamber
VUT_FIT_C236
VUT_FIT_D105
VUT_FIT_Q301
VUT_FIT_E112
VUT_FIT_L207
Hotel_SkalskyDvur_ConferenceRoom2
VUT_FIT_L212
VUT_FIT_L227
Hotel_SkalskyDvur

In [13]:
rir_df['mean_rt60'] = rir_df['room'].apply(lambda row:mean_rt60_dict[row])
#rir_df.to_csv('0812_rir_df.csv')

In [29]:
features_folder = '/scratch/ci411/sonos_rirs/features/081522_20k'
experiments_081822 = glob.glob(os.path.join(features_folder, "081822_*"))
experiments_081822.sort()
room_rir_df = rir_df[['room', 'mean_rt60']]

for exp in experiments_081822:
    csv_path = os.path.join(features_folder, exp, 'feature_df.csv')
    exp_df = pd.read_csv(csv_path)
    #print(exp_df.columns)
    #exp_df = exp_df.merge(room_rir_df, on=['room'], how='left')
    #exp_df = exp_df.rename(mapper={"mean_rt60":"rt60"}, axis='columns')
    #exp_df.to_csv(csv_path, index=False)

Index(['file_speech', 'length (s)', 'file_rir', 'room', 'dataset', 'vol',
       'snr', 'clip_label', 'file_clip', 'file_feature', 'split', 'rt60'],
      dtype='object')
Index(['file_speech', 'length (s)', 'file_rir', 'room', 'dataset', 'vol',
       'snr', 'clip_label', 'file_clip', 'file_feature', 'split', 'rt60'],
      dtype='object')
Index(['file_speech', 'length (s)', 'file_rir', 'room', 'dataset', 'vol',
       'snr', 'clip_label', 'file_clip', 'file_feature', 'split', 'rt60'],
      dtype='object')
Index(['file_speech', 'length (s)', 'file_rir', 'room', 'dataset', 'vol',
       'snr', 'clip_label', 'file_clip', 'file_feature', 'split', 'rt60'],
      dtype='object')
Index(['file_speech', 'length (s)', 'file_rir', 'room', 'dataset', 'vol',
       'snr', 'clip_label', 'file_clip', 'file_feature', 'split', 'rt60'],
      dtype='object')
Index(['file_speech', 'length (s)', 'file_rir', 'room', 'dataset', 'vol',
       'snr', 'clip_label', 'file_clip', 'file_feature', 'split', 'rt60

In [42]:
AUDIO_OUT = '/scratch/ci411/sonos_rirs/reverb_audio'
FEATURES_OUT = '/scratch/ci411/sonos_rirs/features/081522_20k'

room_rir_df = rir_df[['room', 'mean_rt60']]
#route to audio csv and load
audio_name = '081522_20K'
audio_path = os.path.join(AUDIO_OUT, audio_name)
audio_df = pd.read_csv(os.path.join(audio_path, 'audio_df.csv'))

In [52]:
#audio_df = audio_df.merge(room_rir_df, on='room', how='left')\
#                   .rename(mapper={"mean_rt60":"rt60"}, axis='columns')

#audio_df.to_csv(os.path.join(audio_path, 'audio_df.csv'), index=False)

In [53]:
audio_df = pd.read_csv(os.path.join(audio_path, 'audio_df.csv'))
audio_df.columns

Index(['file_speech', 'length (s)', 'file_rir', 'room', 'dataset', 'vol',
       'snr', 'clip_label', 'file_clip', 'rt60'],
      dtype='object')